In [1]:
import python_functions
%load_ext autoreload
%autoreload 2

In [2]:
pilote = "AdrienM"
main_path  ="/Users/Adrien/Documents/paramoteur/syride/analyze_traces/syride_traces/"

In [5]:
python_functions.get_syride_traces(path=main_path, pilot=pilote)

Dossiers trouvés, 3 trace(s) déjà téléchargée(s).
Scrolling...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 58 nouvelles traces trouvées.
Réupération des liens (total 58)...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 Téléchargement des traces (total 58)...
0 Archive extraite dans le répertoire : 2034397
1 Archive extraite dans le répertoire : 2093218
2 Archive extraite dans le répertoire : 2002314
3 Archive extraite dans le répertoire : 2035366
4 Archive extraite dans le répertoire : 2109680
5 Archive extraite dans le répertoire : 2069403
6 Archive extraite dans le répertoire : 2060033
7 Archive extraite dans le répertoire : 1999448
8 Archive extraite dans le répertoire : 2114592
9 Archive extraite dans le répertoire : 1974182
10 Archive extraite dans le répertoire : 1982389
11 Archive extraite dans le répertoire : 1921994
12 Archive extraite dans le répertoire 